In [18]:
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from google.colab import files
uploaded = files.upload()

Saving обработанные данные для анализа без векторизации.csv to обработанные данные для анализа без векторизации.csv


In [43]:
df = pd.read_csv('обработанные данные для анализа без векторизации.csv')
df

,URLs,Headline,Label,Amount of Adverbs,Body_02,Polarity,Subjectivity
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,1,61,image copyright getty imageson sunday morning ...,0.040461,0.415513
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,1,10,london reuters last flag fly comedy drama viet...,0.072048,0.400964
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,1,51,feud break public view last week mr corker say...,0.101149,0.381250
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,1,1,mexico city reuters egypt cheiron holding limi...,0.093254,0.330952
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,1,1,country singer jason aldean perform las vegas ...,0.043561,0.408333
...,...,...,...,...,...,...,...
3983,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,0,1,vietnam great danger must publish tell army go...,0.800000,0.750000
3984,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,0,11,trend watch reader think story fact add two ce...,0.300000,0.510870
3985,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,2,trump jr soon give minute speech reader think ...,0.000000,0.000000
3986,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,1,13,shanghai reuters china say plan accept datum o...,0.069894,0.326308


In [44]:
# достаем домены сайтов
pattern = 'https?://([\w.]+)/'
df['URLs'] = df.URLs.str.extract(pattern)

In [45]:
df

,URLs,Headline,Label,Amount of Adverbs,Body_02,Polarity,Subjectivity
0,www.bbc.com,Four ways Bob Corker skewered Donald Trump,1,61,image copyright getty imageson sunday morning ...,0.040461,0.415513
1,www.reuters.com,Linklater's war veteran comedy speaks to moder...,1,10,london reuters last flag fly comedy drama viet...,0.072048,0.400964
2,www.nytimes.com,Trump’s Fight With Corker Jeopardizes His Legi...,1,51,feud break public view last week mr corker say...,0.101149,0.381250
3,www.reuters.com,Egypt's Cheiron wins tie-up with Pemex for Mex...,1,1,mexico city reuters egypt cheiron holding limi...,0.093254,0.330952
4,www.cnn.com,Jason Aldean opens 'SNL' with Vegas tribute,1,1,country singer jason aldean perform las vegas ...,0.043561,0.408333
...,...,...,...,...,...,...,...
3983,beforeitsnews.com,CNN and Globalist Exposed - Steve Quayle and A...,0,1,vietnam great danger must publish tell army go...,0.800000,0.750000
3984,beforeitsnews.com,Trends to Watch,0,11,trend watch reader think story fact add two ce...,0.300000,0.510870
3985,beforeitsnews.com,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,2,trump jr soon give minute speech reader think ...,0.000000,0.000000
3986,www.reuters.com,China to accept overseas trial data in bid to ...,1,13,shanghai reuters china say plan accept datum o...,0.069894,0.326308


In [6]:
uploaded = files.upload()

Saving модель_для_анализа_заголовка.h5 to модель_для_анализа_заголовка.h5


In [47]:
# анализируем заголовки

model_for_headline = keras.models.load_model('модель_для_анализа_заголовка.h5')

oh_headlines = [one_hot(words, 5000) for words in df['Headline']]
x = pad_sequences(oh_headlines, padding='pre', maxlen=30)

headLines = model_for_headline.predict(x)
df.Headline = headLines

In [48]:
df

,URLs,Headline,Label,Amount of Adverbs,Body_02,Polarity,Subjectivity
0,www.bbc.com,1.0,1,61,image copyright getty imageson sunday morning ...,0.040461,0.415513
1,www.reuters.com,1.0,1,10,london reuters last flag fly comedy drama viet...,0.072048,0.400964
2,www.nytimes.com,1.0,1,51,feud break public view last week mr corker say...,0.101149,0.381250
3,www.reuters.com,0.0,1,1,mexico city reuters egypt cheiron holding limi...,0.093254,0.330952
4,www.cnn.com,1.0,1,1,country singer jason aldean perform las vegas ...,0.043561,0.408333
...,...,...,...,...,...,...,...
3983,beforeitsnews.com,1.0,0,1,vietnam great danger must publish tell army go...,0.800000,0.750000
3984,beforeitsnews.com,1.0,0,11,trend watch reader think story fact add two ce...,0.300000,0.510870
3985,beforeitsnews.com,0.0,0,2,trump jr soon give minute speech reader think ...,0.000000,0.000000
3986,www.reuters.com,1.0,1,13,shanghai reuters china say plan accept datum o...,0.069894,0.326308


In [52]:
# one hot representation of words
oh_text = [one_hot(words, 5000) for words in df.Body_02]
embedded_doc = pad_sequences(oh_text, padding='pre', maxlen=800)

In [53]:
df_text = pd.DataFrame(embedded_doc)

array([[   0,    0,    0, ..., 3483, 4325, 4791],
       [   0,    0,    0, ..., 4480, 3030, 1947],
       [   0,    0,    0, ..., 1065,  720, 1982],
       ...,
       [   0,    0,    0, ..., 3823, 3289, 1348],
       [   0,    0,    0, ..., 2884,  340, 3868],
       [   0,    0,    0, ..., 1480, 1540, 2153]], dtype=int32)

In [56]:
df_new = pd.concat([df.drop(['Body_02'], axis = 1), df_text], axis = 1)

In [57]:
df_new

,URLs,Headline,Label,Amount of Adverbs,Polarity,Subjectivity,0,1,2,3,...,790,791,792,793,794,795,796,797,798,799
0,www.bbc.com,1.0,1,61,0.040461,0.415513,0,0,0,0,...,4717,1184,4736,90,1878,1554,2607,3483,4325,4791
1,www.reuters.com,1.0,1,10,0.072048,0.400964,0,0,0,0,...,4814,2122,2817,3837,117,4149,4044,4480,3030,1947
2,www.nytimes.com,1.0,1,51,0.101149,0.381250,0,0,0,0,...,3837,480,4350,720,2046,1766,2888,1065,720,1982
3,www.reuters.com,0.0,1,1,0.093254,0.330952,0,0,0,0,...,3235,2042,3591,2200,1858,3002,4590,3329,4214,1630
4,www.cnn.com,1.0,1,1,0.043561,0.408333,0,0,0,0,...,122,2211,540,3941,450,2675,881,3376,2216,4698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983,beforeitsnews.com,1.0,0,1,0.800000,0.750000,0,0,0,0,...,0,894,231,2757,43,4247,3637,2047,2700,2269
3984,beforeitsnews.com,1.0,0,11,0.300000,0.510870,0,0,0,0,...,4224,354,4937,811,4886,4524,1400,3021,2076,897
3985,beforeitsnews.com,0.0,0,2,0.000000,0.000000,0,0,0,0,...,4264,3415,3831,1756,3543,607,2171,3823,3289,1348
3986,www.reuters.com,1.0,1,13,0.069894,0.326308,0,0,0,0,...,3459,1619,2864,1584,3446,4480,171,2884,340,3868


In [66]:
# x = pd.DataFrame(create_num(df['Headline'].values))

In [ ]:
# data.info()

In [9]:
# data.dropna(inplace=True, axis=0)

In [10]:
# # taking only the name of website from the URLs
# pattern = 'https?://([\w.]+)/'
# data['Website'] = data.URLs.str.extract(pattern)
# data.drop('URLs', axis=1, inplace=True)

In [17]:
# lm = WordNetLemmatizer()
# def clean_text(column):
#     corpus = []
#     for value in column:
#         value = value.lower()
#         value = re.sub('[^a-z]', ' ', value)
#         value = value.split()
#         value = [lm.lemmatize(word) for word in value\
#                  if word not in set(stopwords.words('english'))]
#         corpus.append(' '.join(value))
#     return corpus    

# data['Text'] = data['Headline'] + ' ' + data['Body']
# data['Text'] = clean_text(data.Text)

In [ ]:
# list to store length of words in each news
# len_list = []
# for w in df.Body_02:
#     w = w.split()
#     len_list.append(len(w))

# print('Summary of word length :')
# pd.Series(len_list).describe()

In [74]:
# initializing model
model = Sequential()

model.add(Embedding(5000, 100, input_length=804))
model.add(LSTM(150))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

## пока без домена новости

In [75]:
x = np.array(df_new.drop(['Label', 'URLs'], axis = 1))
y = df_new.Label

# splitting the dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=3)

In [76]:
x

array([[ 1.00000000e+00,  6.10000000e+01,  4.04608342e-02, ...,
         3.48300000e+03,  4.32500000e+03,  4.79100000e+03],
       [ 1.00000000e+00,  1.00000000e+01,  7.20476190e-02, ...,
         4.48000000e+03,  3.03000000e+03,  1.94700000e+03],
       [ 1.00000000e+00,  5.10000000e+01,  1.01148937e-01, ...,
         1.06500000e+03,  7.20000000e+02,  1.98200000e+03],
       ...,
       [ 0.00000000e+00,  2.00000000e+00,  0.00000000e+00, ...,
         3.82300000e+03,  3.28900000e+03,  1.34800000e+03],
       [ 1.00000000e+00,  1.30000000e+01,  6.98938696e-02, ...,
         2.88400000e+03,  3.40000000e+02,  3.86800000e+03],
       [ 1.00000000e+00,  2.10000000e+01, -4.55291005e-02, ...,
         1.48000000e+03,  1.54000000e+03,  2.15300000e+03]])

In [77]:
# fitting the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
47/47 [==============================] - 121s 3s/step - loss: 0.4192 - accuracy: 0.8225 - val_loss: 0.1643 - val_accuracy: 0.9378
Epoch 2/10
47/47 [==============================] - 115s 2s/step - loss: 0.0874 - accuracy: 0.9753 - val_loss: 0.0833 - val_accuracy: 0.9729
Epoch 3/10
47/47 [==============================] - 115s 2s/step - loss: 0.0186 - accuracy: 0.9936 - val_loss: 0.0763 - val_accuracy: 0.9739
Epoch 4/10
47/47 [==============================] - 114s 2s/step - loss: 0.0037 - accuracy: 0.9997 - val_loss: 0.0787 - val_accuracy: 0.9779
Epoch 5/10
47/47 [==============================] - 114s 2s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0874 - val_accuracy: 0.9799
Epoch 6/10
47/47 [==============================] - 118s 3s/step - loss: 4.8775e-04 - accuracy: 1.0000 - val_loss: 0.1032 - val_accuracy: 0.9759
Epoch 7/10
47/47 [==============================] - 115s 2s/step - loss: 3.2185e-04 - accuracy: 1.0000 - val_loss: 0.1122 - val_accuracy: 0.9769
Epoch

In [78]:
from sklearn.metrics import confusion_matrix, accuracy_score
pred = model.predict(x_test)

In [36]:
pred

array([[9.99962091e-01],
       [9.99973536e-01],
       [8.79628205e-05],
       [6.04607522e-01],
       [2.32221846e-05],
       [9.99959707e-01],
       [9.98983026e-01],
       [9.99921679e-01],
       [9.99933004e-01],
       [4.62877724e-05],
       [8.89857838e-05],
       [9.99966502e-01],
       [2.53215039e-05],
       [9.99965429e-01],
       [9.99921441e-01],
       [9.99958396e-01],
       [9.99945641e-01],
       [4.82350588e-04],
       [2.52740483e-05],
       [9.10330564e-05],
       [7.89617843e-05],
       [6.04607522e-01],
       [6.77088829e-05],
       [2.12297309e-05],
       [7.90473787e-05],
       [9.99668002e-01],
       [9.99940276e-01],
       [9.99889791e-01],
       [9.99961376e-01],
       [5.94913960e-04],
       [9.99790490e-01],
       [9.99948204e-01],
       [6.69975125e-05],
       [9.99961197e-01],
       [9.99934137e-01],
       [6.35303441e-05],
       [9.99957800e-01],
       [3.47835849e-05],
       [1.40130520e-04],
       [2.27291348e-05],


In [79]:
accuracy_score(y_test, np.around(pred))

0.9779338014042126

In [80]:
model.save('финальная_модель.h5')

In [82]:
files.download("финальная_модель.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>